In this Jupyter Notebook we will code from scratch a CNN

In [ ]:
### Imports ###

import numpy as np
from scipy import signal
from tqdm import tqdm

np.random.seed(0)  # For reproductibility

I - Mother class

In [ ]:
class layer :

    def __init__(self) :

        self.input = None
        self.output = None

    def forward(self, input) :
        pass

    def backward(self, grad, eta) :    # eta : learning rate
        pass

II - Dense layer

In [ ]:
class dense_layer(layer) :

    def __init__(self,
                 nb_inputs, 
                 nb_neurones, 
                 weights_initializer = "xavier_norm", 
                 biases_initializer = "xavier_norm",
                 w_std_dev_init = 1,
                 w_mean_init = 0,
                 w_low_uni_init = -1,
                 w_high_uni_init = 1,
                 b_std_dev_init = 1,
                 b_mean_init = 0,
                 b_low_uni_init = -1,
                 b_high_uni_init = 1,
                 optimizer = "GD",
                 moment_gd_param = 0.2
                 ) :
        
        self.nb_neurones = nb_neurones

        if optimizer not in ["GD", "moment_GD", "SGD"] :

            raise TypeError("The optimizer should be in ['GD', 'moment_GD', 'SGD']")
        
        self.optimizer = optimizer

        if optimizer == "moment_GD" :

            self.moment_gd_param = moment_gd_param

        if weights_initializer not in ["zero", "rndu", "rndn", "xavier_uni", "xavier_norm", "He"] :

            raise TypeError("The initializers should be in ['zero', 'rndu', 'rndn', 'xavier_uni', 'xavier_norm', 'He']")

        if biases_initializer not in ["zero", "rndu", "rndn", "xavier_uni", "xavier_norm", "He"] :

            raise TypeError("The initializers should be in ['zero', 'rndu', 'rndn', 'xavier_uni', 'xavier_norm', 'He']")
        
        if weights_initializer == "zero" :
            self.weights = np.zeros((nb_neurones, nb_inputs))

        if weights_initializer == "rndu" :
            self.weights = np.random.uniform(low = w_low_uni_init, high = w_high_uni_init, size = (nb_neurones, nb_inputs))

        if weights_initializer == "rndn" :
            self.weights = w_mean_init + w_std_dev_init*np.random.standard_normal(size = (nb_neurones, nb_inputs))

        if weights_initializer == "xavier_uni" :
            r = np.sqrt(6/(nb_neurones + nb_inputs))
            self.weights = np.random.uniform(low = -r, high = r, size = (nb_neurones, nb_inputs))

        if weights_initializer == "xavier_norm" :
            sigma = np.sqrt(6/(nb_neurones + nb_inputs))
            self.weights = sigma*np.random.standard_normal(size = (nb_neurones, nb_inputs))

        if weights_initializer == "He" :
            sigma = np.sqrt(2/nb_inputs)
            self.weights = sigma*np.random.standard_normal(size = (nb_neurones, nb_inputs))
        
        if biases_initializer == "zero" :
            self.biases = np.zeros(nb_neurones)

        if biases_initializer == "rndu" :
            self.biases = np.random.uniform(low = b_low_uni_init, high = b_high_uni_init, size = (nb_neurones, nb_inputs))

        if biases_initializer == "rndn" :
            self.biases = b_mean_init + b_std_dev_init*np.random.standard_normal(size = nb_neurones)

        if biases_initializer == "xavier_uni" :
            r = np.sqrt(6/(nb_neurones + nb_inputs))
            self.biases = np.random.uniform(low = -r, high = r, size = nb_neurones)

        if biases_initializer == "xavier_norm" :
            sigma = np.sqrt(6/(nb_neurones + nb_inputs))
            self.biases = sigma*np.random.standard_normal( size = nb_neurones)

        if biases_initializer == "He" :
            sigma = np.sqrt(2/nb_inputs)
            self.biases = sigma*np.random.standard_normal( size = nb_neurones)


    def forward(self, input) :

        self.input = input

        return np.dot(self.weights, self.input) + self.biases

    def backward(self, grad, eta) :

        weights_grad = np.dot(np.reshape(grad,(grad.size,1)), np.reshape(self.input,(1,self.input.size)))

        if self.optimizer == "GD" : 

            self.weights -= eta*weights_grad      # weights update
            self.biases -= eta*grad             # biases update

        if self.optimizer == "moment_GD" :
            
            self.momentum = []    #Momentum for the weights and the biases in this order

            gamma = self.moment_gd_param

            if len(self.momentum) == 0 :
                self.momentum.append(weights_grad)
                self.momentum.append(grad)
            
            else :
                self.momentum.append(gamma*self.momentum[-2] + (1-gamma)*weights_grad)
                self.momentum.append(gamma*self.momentum[-2] + (1-gamma)*grad)
                self.momentum = self.momentum[-2:]
            
            self.weights -= eta*self.momentum[-2]      # weights update
            self.biases -= eta*self.momentum[-1]       # biases update
    
        return np.dot(self.weights.T, grad)

III - Convolutionnal layer

In [ ]:
class convolutionnal_layer(layer) :

    def __init__(self,
                 input_shape,
                 kernel_size,
                 depth,
                 optimizer = "GD",
                 moment_gd_param = 0.2
                 ) :


        # We first deal with the dimension of our images, kernes and features

        input_depth, input_height, input_width =  input_shape

        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)

        if optimizer not in ["GD", "moment_GD", "SGD"] :

            raise TypeError("The optimizer should be in ['GD', 'moment_GD', 'SGD']")
        
        self.optimizer = optimizer

        if optimizer == "moment_GD" :

            self.moment_gd_param = moment_gd_param

        # He initialization for ReLU activation
        
        input_size = np.prod(np.array(input_shape))
        sigma = np.sqrt(2/input_size)
        
        self.kernels = sigma*np.random.standard_normal( size = self.kernels_shape)
        self.biases = sigma*np.random.standard_normal(size = self.output_shape)

    def forward(self, input) :

        self.input = input
        self.output = np.copy(self.biases)
        
        for i in range(self.depth) :
            for j in range(self.input_depth) :
            
                self.output[i] += signal.correlate2d(self.input[j], self.kernels[i,j], 'valid')
                
        return self.output

    def backward(self, grad, eta) :

        kernels_grad = np.zeros(self.kernels_shape)
        input_grad = np.zeros(self.input_shape)

        for i in range(self.depth) :
            for j in range(self.input_depth) :

                kernels_grad[i,j] = signal.correlate2d(self.input[j], grad[i], 'valid')
                input_grad[j] +=  signal.convolve2d(grad[i], self.kernels[i,j], 'full')

        if self.optimizer == "GD" :

            self.kernels -= eta * kernels_grad
            self.biases -= eta * grad

        if self.optimizer == "moment_GD" :

            self.momentum = []    #Momentum for the weights and the biases in this order

            gamma = self.moment_gd_param

            if len(self.momentum) == 0 :
                self.momentum.append(kernels_grad)
                self.momentum.append(grad)
            
            else :
                self.momentum.append(gamma*self.momentum[-2] + (1-gamma)*kernels_grad)
                self.momentum.append(gamma*self.momentum[-2] + (1-gamma)*grad)
                self.momentum = self.momentum[-2:]
            
            self.kernels -= eta*self.momentum[-2]      # kernels update
            self.biases -= eta*self.momentum[-1]       # biases update

        return input_grad


III - Activation layer

In [ ]:
class sigmoid_activation_layer(layer) :

    def __init__(self) :

        self.activ_func = lambda x : 1/(1 + np.exp(-x))
        self.derivative = lambda x : 1/(1 + np.exp(-x))*(1 - 1/(1 + np.exp(-x)))

    def forward(self, input) :

        self.input = input
        return self.activ_func(self.input)

    def backward(self, grad, eta) :

        return np.multiply(grad, self.derivative(self.input))

class softmax_activation_layer(layer) : 

    def forward(self,input) : 

        self.output = np.exp(input)/np.sum(np.exp(input))
        
        return np.clip(self.output, 10e-7, 1 - 10e-7)

    def backward(self,grad,eta) : 
        
        return np.dot(grad, self.output)*self.output + np.multiply(grad,self.output)
    
class tanh_activation_layer(layer) : 

    def __init__(self) :

        self.activ_func = lambda x : np.tanh(x)
        self.derivative = lambda x : 1 - np.tanh(x)**2

    def forward(self, input) :

        self.input = input
        return self.activ_func(self.input)

    def backward(self, grad, eta) :

        return np.multiply(grad, self.derivative(self.input))

class ReLU_activation_layer(layer) :

    def __init__(self) :

        self.activ_func = lambda x : np.maximum(0,x)

        def relu_derivative(x) :

            x[x<=0] = 0
            x[x>0] = 1 

            return x

        self.derivative = relu_derivative

    def forward(self, input) :

        self.input = input
        return self.activ_func(self.input)

    def backward(self, grad, eta) :

        return np.multiply(grad, self.derivative(self.input))

class special_tanh_activation_layer1D(layer) : 

    def __init__(self,epsilon) :

        self.epsilon = epsilon
        self.activ_func = lambda x : np.tanh(x)
        self.derivative = lambda x : 1 - np.tanh(x)**2

    def forward(self, input) :

        self.input = input
        return 709*self.activ_func(self.epsilon + self.input/np.mean(self.input))

    def backward(self, grad, eta) :
        
        mu = np.mean(self.input)
        print(f"softmax grad mean : ")
        output = np.zeros(grad.shape)

        for i in range(grad.size) :
            
            delt = 709*(grad.size*mu**2)**(-1)*self.derivative(self.epsilon + self.input/mu)
            
            for j in range(grad.size) :

                if i != j :

                    delt[i] *= -self.input[j]

                else :

                    delt[i] *= grad.size*mu - self.input[j]

            output[i][0] += np.dot(np.transpose(np.reshape(grad, (grad.size,1))), delt)

        return output

IV - Reshape layer

In [ ]:
class reshape_layer(layer) :

    def __init__(self, input_shape, output_shape) :

        self.input_shape = input_shape
        self.output_shape = output_shape

    def forward(self, input) :

        return np.reshape(input, self.output_shape)

    def backward(self, grad, eta) :

        return np.reshape(grad, self.input_shape)

V - Pooling layer

In [ ]:
class avg_pool_layer(layer) :

    def __init__(self, input_shape, kernel_size) :


        # We first deal with the dimension of our images, kernes and features

        input_depth, input_height, input_width =  input_shape

        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (input_depth, input_height - kernel_size + 1, input_width - kernel_size + 1)

        # We iniatlize the kernel 

        self.kernels = kernel_size**(-1)*np.ones((input_depth,kernel_size,kernel_size))

    def forward(self, input) :

        self.input = input
        self.output = np.zeros(self.output_shape)

        for i in range(self.input_depth) :

            self.output[i] += signal.correlate2d(self.input[i], self.kernels[i], 'valid')

        return self.output

    def backward(self, grad, eta) :

        input_grad = np.zeros(self.input_shape)

        for i in range(self.input_depth) :

            input_grad[i] +=  signal.convolve2d(grad[i], self.kernels[i], 'full')

        return input_grad

VI- Loss layer

In [ ]:
class cross_entropy : 

    def __init__(self, y_pred, y_true) : 

        self.y_pred = np.clip(y_pred, 10e-7, 1 - 10e-7)
        self.y_true = y_true

    def compute(self) :

        error = -np.dot(np.transpose(np.log(self.y_pred)),self.y_true)
        
        return error[0]
    
    def grad(self) : 

        grad = -np.reshape(self.y_true,(15,))/self.y_pred

        return grad

VII - Normalizing layer

In [ ]:
class normalize_layer1D(layer) : 

    def __init__(self, input_shape, epsilon = 1e-8) :
        
        self.coeff = np.random.randn()
        self.biases = np.zeros(input_shape)
        self.epsilon = epsilon


    def forward(self,input) :
        
        self.mean = np.mean(input)
        self.stdev = np.std(input)
        self.stand_input = (input - self.mean)/(self.stdev + self.epsilon)
         
        return self.coeff*self.stand_input + self.biases

    def backward(self,grad,eta) : 

        # grad = np.reshape(grad, (grad.size,1))  # Just in case

        output = np.zeros(grad.shape)  
        n = grad.size

        jacob = np.zeros((n,n))

        for i in range(n) :
            for j in range(n) : 

                jacob[i,j] *= self.stand_input[i]*self.stand_input[j]
                jacob[i,j] *= -(self.stdev + self.epsilon)/(n*self.stdev)
                jacob[i,j] *= +self.coeff/(n*(self.stdev + self.epsilon))


                if i != j :
                    
                    
                    jacob[i,j] += self.coeff/(n*(self.stdev + self.epsilon))

                else : 

                    jacob[i,j] += self.coeff/(self.stdev + self.epsilon)*(1+n**(-1))



        self.coeff -= np.dot(np.transpose(grad),self.stand_input)
        self.biases -= eta*grad

        return np.dot(jacob, grad)